In [ ]:
#Requirements: nltk, tensorflow, keras, tweepy, numpy, wikipedia
#Importing and Pre-processing data!
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
import random
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
nltk.download('punkt')
nltk.download('wordnet')

words=[]
classes = []
documents = []
ignore_words = ['.', ',', '?', '!', '#', '@']
data_file = open('intents.json').read()
intents = json.loads(data_file)
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #Splitting sentences into smaller phrases!
        wo = nltk.word_tokenize(pattern)
        words.extend(wo)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

#lower casing each word and grouping similar meaning words for the CNN
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))


In [ ]:
##Creating training and testing data!
training = []
output_empty = [0] * len(classes)
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
random.shuffle(training)
training = np.array(training)
#Splitting data into X and Y planes!
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data has been created!")

In [ ]:
#Creating a CNN with 3 layers: first-128, second-64, final-output layer
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=4, verbose=1)
model.save('model.h5', hist)

print("trained model completed!")


In [ ]:
#preparing the model
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import wikipedia
from keras.models import load_model
import json
import random

model = load_model('model.h5')
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))


def clean_up(sentence): #function to link similar words to one word!
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words


def bow(sentence, words, show_details=True):
    sentence_words = clean_up(sentence)
    bag = [0]*len(words)  
    for i in sentence_words:
        for j,k in enumerate(words):
            if k == i: 
                bag[j] = 1
                if show_details:
                    print ("found: %s" % k)
    return(np.array(bag))

def predict_class(sentence, model):
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def get_time():
    from datetime import datetime
    now = datetime.now()
    current_time = now.strftime("%H:%M")
    time = "The time is: "+current_time
    return time

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

def wiki_data(msg):
    result = wikipedia.summary(msg, sentences = 2) 
    result = str(result)
    return result
    
def weather_data():
    pass

In [ ]:
#Connect to twitter api
import tweepy
from tweepy import *

#Access requiements to connect to twitter api!
token1 = ""
token2 = ""
token3 = ""
token4 = ""
auth = tweepy.OAuthHandler(token1, token2)
auth.set_access_token(token3, token4)
# Creating API handler
api = tweepy.API(auth)
try:
    api.verify_credentials()
    print("API Verified!")
except:
    print("Error")

In [ ]:
#Get the chatbot response
import time

name = "LuqmaanKamaar" #Inserting my user name
user = api.get_user(name) #Getting user name as seen by api
ID = user.id_str  #Getting the ID of my username
time.sleep(20) #Waiting for tweepy to reload in order to list current messages!
My_messages = api.list_direct_messages(count=5) #Getting direct messages from api
for messages in reversed(My_messages): #looping through the list from most recent to older
    sender_Id = messages.message_create["sender_id"] #Storing the ID of the recent sender
    text = messages.message_create["message_data"]["text"] #Storing the text of the recent sender
print(sender_Id, text)#Printing the ID and the actual message of the last received DM 

if 'What is the time' in text.lower(): #determining if the user is asking for the time
    api.send_direct_message(sender_Id, get_time())
#elif '1395793448310124548' == sender_Id:
#    api.destroy_direct_message('1395793448310124548')
else:
    api.send_direct_message(sender_Id, chatbot_response(text)) #Otherwise, implementing the trained chatbot!